In [20]:
import gurobipy as gp

In [21]:
# Dados brutos
qtd_fabricas = 5
qtd_clientes = 10
valor_p = 2
mat_custos = [
    [684, 439, 441, 1078, 806, 557, 443, 356, 688, 322],
    [441, 728, 822, 21, 440, 728, 1202, 1121, 934, 1077],
    [791, 576, 574, 1197, 899, 692, 385, 322, 680, 306],
    [700, 974, 1059, 275, 675, 954, 1458, 1380, 732, 1337],
    [217, 375, 483, 641, 324, 503, 558, 470, 859, 424]]

In [22]:
fabricas = ["F{}".format(i+1) for i in range (qtd_fabricas)]
clientes = ["C{}".format(i+1) for i in range (qtd_clientes)]
print(fabricas)
print(clientes)


['F1', 'F2', 'F3', 'F4', 'F5']
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']


In [23]:
#Dicionario com os custos
custos = dict()

for i, fab in enumerate(fabricas):
    for j, cli in enumerate(clientes):
        custos[fab,cli] = mat_custos[i][j]
        
print (custos['F1','C1'])

684


In [24]:
print (custos)

{('F1', 'C1'): 684, ('F1', 'C2'): 439, ('F1', 'C3'): 441, ('F1', 'C4'): 1078, ('F1', 'C5'): 806, ('F1', 'C6'): 557, ('F1', 'C7'): 443, ('F1', 'C8'): 356, ('F1', 'C9'): 688, ('F1', 'C10'): 322, ('F2', 'C1'): 441, ('F2', 'C2'): 728, ('F2', 'C3'): 822, ('F2', 'C4'): 21, ('F2', 'C5'): 440, ('F2', 'C6'): 728, ('F2', 'C7'): 1202, ('F2', 'C8'): 1121, ('F2', 'C9'): 934, ('F2', 'C10'): 1077, ('F3', 'C1'): 791, ('F3', 'C2'): 576, ('F3', 'C3'): 574, ('F3', 'C4'): 1197, ('F3', 'C5'): 899, ('F3', 'C6'): 692, ('F3', 'C7'): 385, ('F3', 'C8'): 322, ('F3', 'C9'): 680, ('F3', 'C10'): 306, ('F4', 'C1'): 700, ('F4', 'C2'): 974, ('F4', 'C3'): 1059, ('F4', 'C4'): 275, ('F4', 'C5'): 675, ('F4', 'C6'): 954, ('F4', 'C7'): 1458, ('F4', 'C8'): 1380, ('F4', 'C9'): 732, ('F4', 'C10'): 1337, ('F5', 'C1'): 217, ('F5', 'C2'): 375, ('F5', 'C3'): 483, ('F5', 'C4'): 641, ('F5', 'C5'): 324, ('F5', 'C6'): 503, ('F5', 'C7'): 558, ('F5', 'C8'): 470, ('F5', 'C9'): 859, ('F5', 'C10'): 424}


In [35]:
m = gp.Model()

#Variaveis de Decisão

var_x = m.addVars(fabricas, clientes, vtype = gp.GRB.BINARY) 
var_y = m.addVars(fabricas, vtype = gp.GRB.BINARY)

#Restrição 1: Cada cliente tem que estar em 1 Fábrica
c1 = m.addConstrs (gp.quicksum(var_x[i, j] for i in fabricas) == 1 for j in clientes)
#Resctrição 2: Garante abertura de até p fábricas
c2 = m.addConstr(gp.quicksum(var_y[i] for i in fabricas) == valor_p)
#Restrição 3: Garante a designição de clientes
c3 = m.addConstrs(var_x[i, j] - var_y[i] <= 0 for i in fabricas for j in clientes)

custo_total = gp.quicksum(custos[i,j]*var_x[i,j]for i in fabricas for j in clientes)
c4 = m.addConstrs(gp.quicksum(custos[i,j]*var_x[i,j]for i in fabricas) - var_R <=0 for j in clientes)
#cria uma expressão que calcula o custo  total
m.setObjective(custo_total , sense = gp.GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 61 rows, 55 columns and 155 nonzeros
Model fingerprint: 0xae49f5b2
Variable types: 0 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 5748.0000000
Presolve time: 0.03s
Presolved: 61 rows, 55 columns, 155 nonzeros
Variable types: 0 continuous, 55 integer (55 binary)

Root relaxation: objective 4.148000e+03, 29 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    4148.0000000 4148.00000  0.00%     -    0s

Explored 0 nodes (29 simplex iterations) in 0.12 seconds
Thread count was 4 (of 4 available processor

In [36]:
#Verifica quantas fábricas foram abertas
for i in fabricas:
    if (round(var_y[i].X) == 1):
        print(i)

F1
F2


In [39]:
for i in fabricas:
    if round(var_y[i].X) == 1:
        
        for j in clientes:
            if round(var_x[i,j].X) == 1:
                print(j + ' ', end = '')
        print(' ')

C2 C3 C6 C7 C8 C9 C10  
C1 C4 C5  
